https://github.com/nyu-mll/jiant/blob/e6d90621ef7e6277d4fd92241fea8b5646352e64/guides/tasks/adding_tasks.md


    1. A (new) task file: jiant/tasks/lib/senteval/mytask.py
    2. The task retrieval library: jiant/tasks/retrieval.py
    3. The evaluation library: jiant/tasks/evaluate/core.py


 exemple : implémentation de udep, panx, ccg
 attention panx, udep sont des benchmarks avec des sous-tâches ... ça doit rien changer en théorie sur cette partie
 jiant/jiant/tasks/lib/
 
 essentiellement: changer la liste de labels, et écrire une fonction pour lire les données, cf panx qui est du NER

In [ ]:
# 2 dans retrieval

TASK_DICT = {
# ...
	"mytask": MyTask,
# ...
}

In [ ]:
# 3 dans evaluate, ajouter ici la tâche

elif isinstance(task, (tasks.UdposTask, tasks.PanxTask, tasks.MyTask)):
        return F1TaggingEvaluationScheme()

In [ ]:
il faut aussi régler la question du téléchargement, expliqué dans le doc adding_task

pour tokenclassif: ajoute juste une couche de projection sur les labels (avec un dropout). 
si on veut qqchose de plus évolué il faut changer ça dans 
jiant/jiant/proj/main/modeling/heads.py

In [ ]:
@JiantHeadFactory.register([TaskTypes.TAGGING])
class TokenClassificationHead(BaseHead):
    def __init__(self, task, hidden_size, hidden_dropout_prob, **kwargs):
        """From RobertaForTokenClassification"""
        super().__init__()
        self.num_labels = len(task.LABELS)
        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.classifier = nn.Linear(hidden_size, self.num_labels)

    def forward(self, unpooled):
        unpooled = self.dropout(unpooled)
        logits = self.classifier(unpooled)
        return logits

In [ ]:
# prototype modification
recurrent_modules = {
    "lstm": nn.LSTM,
    "gru": nn.GRU,
}

lstm_cfg = {
    "hidden_size": 100,
    "bidirectional": False,
    
}

@JiantHeadFactory.register([TaskTypes.TAGGING])
class TokenClassificationHead(BaseHead):
    def __init__(self, task, hidden_size, hidden_dropout_prob, recurrent_layer=None,**kwargs):
        """From RobertaForTokenClassification
        
        adapted to add a recurrent layer on the tokens outputs
        beware that "hidden_size" input is actually transformer output size 
        """
        super().__init__()
        self.num_labels = len(task.LABELS)
        self.dropout = nn.Dropout(hidden_dropout_prob)
        if recurrent_layer:
            # kwargs should have at least its own hidden_size
            #     
            self.rnn = recurrent_modules[recurrent_layer](input_size=hidden_size,**kwargs["rnn_config"])
            output_size = **kwargs["rnn_config"]["hidden_size"]
            self.projection = nn.Linear(output_size, self.num_labels)
            self.classif_type = "recurrent_layer"
        else:
            self.classif_type = "simple"
            self.classifier = nn.Linear(hidden_size, self.num_labels)

    def forward(self, unpooled):
        unpooled = self.dropout(unpooled)
        if self.classif_type =="simple":
            logits = self.classifier(unpooled)
        else:# rnn
            outputs, hn_cn = self.rnn(unpooled)
            logits = self.classifier(outputs)
        return logits

en fait la taskhead reçoit la task d'origine dont tout paramètre défini dans task peut servir à définir 
le forward et la head. 
il suffit de réécrire la task head avec ce qu'on suppose etre la config de la task: 
ici est un model simple ou avec rnn

In [10]:
def test(a=1,b=2):
    return a,b

def init(x,**kwargs):
    print(test(**kwargs["t"]))

init(1,t={"a":3,"b":4})

(3, 4)
